<font color="#CC3D3D"><p>
# Model Tuning (Hyperparameter Optimization)

<img align="left" src="https://i1.wp.com/hugrypiggykim.com/wp-content/uploads/2017/09/hyper-parameter-search.jpg?resize=698%2C242" width=800 height=600 alt="Decision Tree">

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, random_state=0)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

model = KNeighborsClassifier()
#model = DecisionTreeClassifier()
#model = LogisticRegression()
#model = SVC()

<br><font color = "darkgreen">
## 1. Grid Search CV 

< Grid Search 과정 예시 >   
<img align='left' src='http://drive.google.com/uc?export=view&id=1uoEGcdqMVfHsibXrjII-bwgbQi1mWDH0' width=500>   

##### Set the parameters for grid search #####

In [ ]:
# param_grid: dictionary with parameters names as keys and
# lists of parameter settings to try as values

param_grid = {'n_neighbors': range(4,10),
              'weights': ['uniform','distance']}
param_grid

##### Grid search with cross-validation ####

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(model, param_grid, scoring='accuracy', cv=5, n_jobs=-1)

In [ ]:
# grid search is very time-consuming

grid_search.fit(X_train, y_train)

##### Evaluate the model with best parameters ####

In [ ]:
grid_search.score(X_test, y_test), KNeighborsClassifier().fit(X_train, y_train).score(X_test, y_test)

In [ ]:
print("Best parameters: {}".format(grid_search.best_params_))
print("Best CV score: {:.2f}".format(grid_search.best_score_))

In [ ]:
print("Best estimator:\n{}".format(grid_search.best_estimator_))

<br><font color = "darkgreen">
## 2. Random Search CV

< Random Search 과정 예시 >   
<img align='left' src='http://drive.google.com/uc?export=view&id=19m7DTbD1ltuuydM-uAfjTK5X5qP0eO-C' width=500>

##### Set the parameters for random search #####

In [ ]:
param_grid = {'n_neighbors': range(1, 10),
              'weights': ['uniform','distance']}
param_grid

##### Random search with cross-validation ####

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

rand_search = RandomizedSearchCV(model, param_distributions=param_grid, 
                                 scoring='accuracy', n_iter=12, random_state=100)

In [ ]:
rand_search.fit(X_train, y_train)

##### Evaluate the model with best parameters ####

In [ ]:
rand_search.score(X_test, y_test)

In [ ]:
print("Best estimator:\n{}".format(rand_search.best_estimator_))

In [ ]:
print("Best parameters: {}".format(rand_search.best_params_))

<br><font color = "darkgreen">
#### Grid Search 결과와 Random Search 결과 비교 예시 [Bergstra and Bengio(2012)]

<img align='left' src='http://drive.google.com/uc?export=view&id=1qa6Ouqx31a14N8Y5fE48P1aDgUJnNciC' width=600>

Random Search는 Grid Search에 비해 불필요한 반복 수행 횟수를 대폭 줄이면서, 동시에 정해진 간격(grid) 사이에 위치한 값들에 대해서도 확률적으로 탐색이 가능하므로, 최적 hyperparameter 값을 더 빨리 찾을 수 있는 것으로 알려져 있음.

## 3. Bayesian Optimization with Optuna

- Grid Search와 Random Search는 이전까지의 조사 과정에서 얻어진 hyperparameter 값들의 성능 결과에 대한 '사전 지식'이 전혀 반영되어 있지 않기 때문에 비효율적인 요소가 있음.
- 매 회 새로운 hyperparameter 값에 대한 조사를 수행할 시 '사전 지식'을 충분히 반영하면서, 동시에 전체적인 탐색 과정을 체계적으로 수행할 수 있는 방법이 Bayesian Optimization임.   

<img align='left' src='https://github.com/fmfn/BayesianOptimization/raw/master/examples/bo_example.png' width=700>

In [ ]:
#!pip install optuna

##### A simple optimization problem #####

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.arange(-10, 10)
plt.plot(x, (x-2)**2)
plt.show()

In [ ]:
"""
A simple optimization problem:

- Define objective function to be optimized. Let's minimize (x - 2)^2
- Suggest hyperparameter values using trial object. Here, a float value of x is suggested from -10 to 10
- Create a study object and invoke the optimize method over 100 trials
"""

import optuna

def objective(trial):
    x = trial.suggest_uniform('x', -10, 10)
    return (x - 2) ** 2

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params

##### Procedure for optimizing sklearn parameters #####
1. Wrap model training with an objective function and return accuracy
2. Suggest hyperparameters using a trial object
3. Create a study object and execute the optimization

In [ ]:
from sklearn.model_selection import cross_val_score

# 조절할 하이퍼 파라미터와 그 범위를 지정하는 함수 정의
def objective(trial): 
    # optuna.trial.Trial.suggest_categorical() for categorical parameters
    # optuna.trial.Trial.suggest_int() for integer parameters
    # optuna.trial.Trial.suggest_float() for floating point parameters    
    knn_n_neighbors = trial.suggest_int('n_neighbors', 1, 10, step=1)
    knn_weights = trial.suggest_categorical('weights', ['uniform','distance'])

    classifier_obj = KNeighborsClassifier(
        n_neighbors = knn_n_neighbors, 
        weights = knn_weights,    
    )

    score = cross_val_score(classifier_obj, X_train, y_train, cv=5, n_jobs=-1)
    accuracy = score.mean()
    return accuracy

# 최적화 실행
study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=100), direction="maximize")
study.optimize(objective, n_trials=12) 

#최적화 결과 보기
print("Best score:", study.best_value)
print("Best parameters:", study.best_params)

##### Plotting the optimization process #####

In [ ]:
model = KNeighborsClassifier(**study.best_params)
model.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
# 최적화 과정 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

<font color="#CC3D3D"><p>
# End